In [3]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [4]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

# TOKENS = [
#     "AAPL", "GOOG", "MSFT", "TSLA", "NVDA", "AMZN",
#     "FB", "BABA", "CRM", "INTC", "AMD", "PYPL", "ATVI",
#     "EA", "TTD", "MTCH", "ZG", "YELP"
# ]
TOKENS = ["X:BTCUSD"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2018-02-18").date()
TO_DATE = pd.Timestamp("2022-04-11").date()

DATASET_NAME = "BTC"

FULL_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}.csv"
TRAIN_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_train.csv"
TEST_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_test.csv"

In [5]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [6]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [7]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Current date: 2020-05-25
Get DataFrame of shape (3333, 8)
Current date: 2020-06-29
Get DataFrame of shape (3333, 8)
Current date: 2020-08-03
Get DataFrame of shape (3333, 8)
Current date: 2020-09-07
Get DataFrame of shape (3333, 8)
Current date: 2020-10-12
Get DataFrame of shape (3333, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2020-11-16
Get DataFrame of shape (3333, 8)
Current date: 2020-12-21
Get DataFrame of shape (3333, 8)
Current date: 2021-01-25
Get DataFrame of shape (3333, 8)
Current date: 2021-03-01
Get DataFrame of shape (3333, 8)
Current date: 2021-04-05
Get DataFrame of shape (3333, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


...

Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2022-04-12
Get DataFrame of shape (2016, 8)


In [8]:
RENAME_COLUMNS = {
    "v": "volume",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "date"
}
KEEP_COLUMNS = ["volume", "open", "close", "high", "low", "date", "ticker"]

result_df = pd.concat(dfs).rename(RENAME_COLUMNS, axis=1)[KEEP_COLUMNS].reset_index(drop=True)
print(result_df.shape)
result_df

(68676, 7)


,volume,open,close,high,low,date,ticker
0,88.874451,7140.74,7145.34,7162.059278,7132.54,2020-04-19 08:58:00,X:BTCUSD
1,364.228687,7145.34,7160.58,7162.700000,7133.37,2020-04-19 09:13:00,X:BTCUSD
2,473.196157,7160.58,7168.80,7170.000000,7140.68,2020-04-19 09:28:00,X:BTCUSD
3,321.947133,7164.59,7182.02,7205.000000,7144.21,2020-04-19 09:43:00,X:BTCUSD
4,305.291586,7181.91,7175.50,7196.100000,7161.62,2020-04-19 09:58:00,X:BTCUSD
...,...,...,...,...,...,...,...
68671,229.112447,39935.54,39769.78,39968.000000,39680.50,2022-04-11 22:45:00,X:BTCUSD
68672,176.391432,39769.78,39712.40,39863.000000,39579.57,2022-04-11 23:00:00,X:BTCUSD
68673,191.662661,39712.38,39641.48,39756.000000,39565.07,2022-04-11 23:15:00,X:BTCUSD
68674,192.552584,39629.80,39477.91,39710.000000,39445.29,2022-04-11 23:30:00,X:BTCUSD


In [10]:
result_df

,volume,open,close,high,low,date,ticker
0,88.874451,7140.74,7145.34,7162.059278,7132.54,2020-04-19 08:58:00,X:BTCUSD
1,364.228687,7145.34,7160.58,7162.700000,7133.37,2020-04-19 09:13:00,X:BTCUSD
2,473.196157,7160.58,7168.80,7170.000000,7140.68,2020-04-19 09:28:00,X:BTCUSD
3,321.947133,7164.59,7182.02,7205.000000,7144.21,2020-04-19 09:43:00,X:BTCUSD
4,305.291586,7181.91,7175.50,7196.100000,7161.62,2020-04-19 09:58:00,X:BTCUSD
...,...,...,...,...,...,...,...
68671,229.112447,39935.54,39769.78,39968.000000,39680.50,2022-04-11 22:45:00,X:BTCUSD
68672,176.391432,39769.78,39712.40,39863.000000,39579.57,2022-04-11 23:00:00,X:BTCUSD
68673,191.662661,39712.38,39641.48,39756.000000,39565.07,2022-04-11 23:15:00,X:BTCUSD
68674,192.552584,39629.80,39477.91,39710.000000,39445.29,2022-04-11 23:30:00,X:BTCUSD


In [11]:
TEST_RATIO = 0.2
test_size = int(result_df.shape[0] * TEST_RATIO)
data_train = result_df.iloc[:-test_size]
data_test = result_df.iloc[-test_size:]

result_df.to_csv(FULL_RESULT_PATH)
data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)

In [12]:
test_size

13735